In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
df = pd.read_csv('revlog_202012160921.csv')
df.head()

,id,cid,ease,lastIvl,type
0,1592896118009,1592895039663,1,-60,0
1,1592896125289,1592895039663,3,-60,0
2,1592896135539,1592895039663,1,-600,0
3,1592896138743,1592895039663,3,-60,0
4,1592896144633,1592895039663,3,-600,0


In [3]:
df = df.drop(df[df['ease']==0].index)

In [4]:
df['raw_date'] = pd.to_datetime(df['id'],unit='ms')
df['raw_date'] = df['raw_date'].dt.tz_localize('UTC').dt.tz_convert('Asia/Shanghai')
df.head()

,id,cid,ease,lastIvl,type,raw_date
0,1592896118009,1592895039663,1,-60,0,2020-06-23 15:08:38.009000+08:00
1,1592896125289,1592895039663,3,-60,0,2020-06-23 15:08:45.289000+08:00
2,1592896135539,1592895039663,1,-600,0,2020-06-23 15:08:55.539000+08:00
3,1592896138743,1592895039663,3,-60,0,2020-06-23 15:08:58.743000+08:00
4,1592896144633,1592895039663,3,-600,0,2020-06-23 15:09:04.633000+08:00


In [5]:
df['date'] = df['raw_date'].map(lambda x: x-timedelta(days=1) if x.hour < 5 else x)
df['date'] = df['date'].dt.floor('D')
df.drop(df[df['date'].dt.year < 2017].index, inplace=True)
df.drop_duplicates(['cid', 'date'], keep='first', inplace=True)
df.head()

,id,cid,ease,lastIvl,type,raw_date,date
0,1592896118009,1592895039663,1,-60,0,2020-06-23 15:08:38.009000+08:00,2020-06-23 00:00:00+08:00
5,1601963355731,1554031768886,1,-60,0,2020-10-06 13:49:15.731000+08:00,2020-10-06 00:00:00+08:00
6,1601963380268,1554031768887,3,-60,0,2020-10-06 13:49:40.268000+08:00,2020-10-06 00:00:00+08:00
7,1601963394156,1554031768889,4,-60,0,2020-10-06 13:49:54.156000+08:00,2020-10-06 00:00:00+08:00
8,1601963407748,1554031768892,3,-60,0,2020-10-06 13:50:07.748000+08:00,2020-10-06 00:00:00+08:00


In [6]:
df.sort_values(by=['cid', 'id'], inplace=True)
df['used_interval'] = df.date.diff().dt.days
df.loc[df['type']==0, 'used_interval'] = 0

In [7]:
for cid in df.loc[df['used_interval']<0, 'cid']:
    df.drop(df[df.cid == cid].index, inplace=True)

In [8]:
df['p'] = df['ease'].map({1:0, 2:1, 3:1, 4:1})

In [9]:
del df['id']
del df['raw_date']
del df['date']

In [10]:
df['acc_rep'] = 0
df['acc_lapses'] = 0
df['con_rep'] = 0
df['con_lapses'] = 0
df['repeat_long'] = 1
df.head()

,cid,ease,lastIvl,type,used_interval,p,acc_rep,acc_lapses,con_rep,con_lapses,repeat_long
5,1554031768886,1,-60,0,0.0,0,0,0,0,0,1
579,1554031768886,1,4,1,3.0,0,0,0,0,0,1
814,1554031768886,4,1,1,1.0,1,0,0,0,0,1
2739,1554031768886,3,4,1,4.0,1,0,0,0,0,1
8095,1554031768886,3,10,1,11.0,1,0,0,0,0,1


In [11]:
cols = ['cid', 'type', 'repeat_long', 'acc_lapses', 'con_lapses', 'acc_rep', 'con_rep', 'used_interval', 'lastIvl', 'ease', 'p']
df = df[cols]
df.head()

,cid,type,repeat_long,acc_lapses,con_lapses,acc_rep,con_rep,used_interval,lastIvl,ease,p
5,1554031768886,0,1,0,0,0,0,0.0,-60,1,0
579,1554031768886,1,1,0,0,0,0,3.0,4,1,0
814,1554031768886,1,1,0,0,0,0,1.0,1,4,1
2739,1554031768886,1,1,0,0,0,0,4.0,4,3,1
8095,1554031768886,1,1,0,0,0,0,11.0,10,3,1


In [12]:
def get_feature(x):
    for idx, log in enumerate(x.itertuples()):
        if idx == x.shape[0] - 1:
            break
        x.iloc[idx + 1, 2] = x.iloc[idx, 2] + 1
        if log.p == 1:
            x.iloc[idx + 1, 3] = x.iloc[idx, 3]
            x.iloc[idx + 1, 4] = 0
            x.iloc[idx + 1, 5] = x.iloc[idx, 5] + 1
            x.iloc[idx + 1, 6] = x.iloc[idx, 6] + 1
        else:
            x.iloc[idx + 1, 3] = x.iloc[idx, 3] + 1
            x.iloc[idx + 1, 4] = x.iloc[idx, 4] + 1
            x.iloc[idx + 1, 5] = x.iloc[idx, 5]
            x.iloc[idx + 1, 6] = 0
    return x
df = df.groupby('cid').apply(get_feature)

In [13]:
df.to_csv('revlog_202012160921_clear.csv', index=False)